# Tutorial

This example gives a general overview on how to pre- and postprocess ASCOT5 simulations.

1. <a href='#stepbystep'>First simulation: step-by-step</a>
2. <a href='#contents'>Contents of the HDF5 file</a>
3. <a href='#ascotpy'>Python interface to libascot.so</a>
4. <a href='#inputgen'>Input generation</a>
5. <a href='#postprocess'>Post processing</a>
6. <a href='#simulation'>Live simulations</a>

<a id='stepbystep'></a>

## First simulation: step-by-step

Go to your ascot5 folder where you compiled the code and type `ipython3` to begin this tutorial. Then repeat these steps:

1. All pre- and post-processing is done via `Ascot` object.
To create a new ASCOT5 data file, use `create=True`.

In [ ]:
import numpy as np
from a5py import Ascot

a5 = Ascot("ascot.h5", create=True)
print("File created")

2. The following lines initialize test data. We will go through the input generation in detail later.

In [ ]:
# Use pre-existing template to create some input data
a5.data.create_input("options tutorial")
a5.data.create_input("bfield analytical iter circular")
a5.data.create_input("wall rectangular")
a5.data.create_input("plasma flat")

# Create electric field and markers by giving input parameters explicitly
from a5py.ascot5io.marker import Marker
mrk = Marker.generate("gc", n=100, species="alpha")
mrk["energy"][:] = 3.5e6
mrk["pitch"][:]  = 0.99 - 1.98 * np.random.rand(100,)
mrk["r"][:]      = np.linspace(6.2, 8.2, 100)
a5.data.create_input("gc", **mrk)
a5.data.create_input("E_TC", exyz=np.array([0,0,0])) # Zero electric field

# Create dummy input for the rest
a5.data.create_input("N0_3D")
a5.data.create_input("Boozer")
a5.data.create_input("MHD_STAT")
print("Inputs initialized")

3. **EITHER** close the ipython session and edit options on terminal (opens a text editor):

   `a5editoptions ascot.h5`

   Scroll down to "End conditions" and set `ENDCOND_MAX_MILEAGE = 0.5e-2`. Save and close the editor. When prompted, set "Fast" as a description.

   **OR** set the options in ipython:

In [ ]:
name = a5.data.options.active.new(ENDCOND_MAX_MILEAGE=0.5e-2, desc="Fast")
a5.data.options[name].activate()
print("Options updated")

4. Now execute `ascot5_main` which should take less than 10 seconds.
    
   **EITHER** in terminal:
   
   `./ascot5_main --d="Hello world!"`

   **OR** in ipython:

In [ ]:
import subprocess
subprocess.run(["./ascot5_main", "--d=\"Hello world!\""])
print("Simulation completed")

5. Open `ipython` again to read the data and plot marker endstates.

   Hint: You can add the line `from a5py import Ascot` to your ipython config file (`~/.ipython/profile_default/ipython_config.py`) so that it is automatically called in beginning of every ipython session:

   <pre>
        c = get_config()
        c.InteractiveShellApp.exec_lines = [
            '%load_ext autoreload',
            '%autoreload 2',
            'import numpy as np',
            'import scipy',
            'import matplotlib as mpl',
            'import matplotlib.pyplot as plt',
            'from a5py import Ascot',
        ]
   </pre>

In [ ]:
from a5py import Ascot
import numpy as np
import matplotlib.pyplot as plt

a5 = Ascot("ascot.h5")
a5.data.ls(show=True) # Print summary of the data within this file
mil = a5.data.active.getstate("mileage", state="end") # How much time passed for each marker
print("Average mileage: %0.5f" % (np.mean(mil)))

# Plots markers' final (R, z) coordinates and wall contour
ax = plt.figure().add_subplot(1,1,1)
a5.data.active.plotstate_scatter("end r", "end z", axes=ax)
plt.show(block=False)

6. That's it! Now you have prepared inputs, ran ASCOT5, and accessed the simulation output.

   For actual simulations each step is more involving, but the basic premise was illustrated here. ASCOT5 development aims to integrate most input generation and post-processing tools so that they can be accessed via an `Ascot` object. Therefore, it is a good idea to always check from the documentation if there is an existing tool available.

   Next in this tutorial, we will go through each step in detail. However, don't forget to try ASCOT5 GUI for fast and easy access to the data (type `a5gui ascot.h5` in terminal to open it).

<a id='contents'></a>

## Contents of the HDF5 file

ASCOT5 stores the data in a HDF5 file.
The file contains **all** inputs necessary to (re)run the simulation and the simulation output.
The format supports multiple inputs and multiple outputs so that all data relevant for a single study can be stored in a single file (but remember to make backups!).
To separate inputs and simulations from one another, each input and each simulation is assigned a quasi-unique identifier (QID) which is string of ten numbers from 0-10.

How exactly the data is stored in a file is not relevant, as the data is accessed via `Ascot` object.
Or to be more precise, the contents of the file is accessed and modified via `Ascot.data` attribute which is an `Ascot5IO` object.

The contents of the file can be quickly viewed with `ls()` method (GUI is also very good for this purpose).

In [ ]:
# In ipython terminal:
from a5py import Ascot
a5 = Ascot("ascot.h5")
info = a5.data.ls()

The `data` attribute provides a *treeview* of the contents.
At the top level are *input parent groups*, e.g. `bfield`, and *result groups*.
Each input parent group contains all inputs in that category, e.g. `bfield` contains every magnetic field input.
The one that is going to be used in a simulation is marked with an *active* flag.
Again, `ls()` method can be used to view the contents of input parent groups.

In [ ]:
info = a5.data.options.ls()

As can be seen, each input has a QID (the quasi-unique identifier), date when it was created, user-given description, and name that has a format \<inputtype\>_\<qid\>.
Note that when we used `a5editoptions` to modify options, the old options were preserved.

Objects representing the inputs can be accessed via their name, qid, or tag which is the first word in the description.
Both attribute-like and dictionary-like referencing is supported.

In [ ]:
# These all point to same object (note that this cell fails if commented lines are uncommented without using proper QID)
#a5.data.options.q1234567890       # Ref by QID, note "q" prefix
a5.data.options.FAST               # Ref by tag, note that it is always all caps, no special symbols allowed and max 10 characters
#a5.data.options.Opt_1234567890    # Ref by name
a5.data.options.active             # Ref to options input that is currently active and will be used in the next simulation
#a5.data["options"]["q1234567890"] # Dictionary-like access

Each input object (as well as result group) has methods to access its meta data and alter the description (and, hence, tag).
The inbut objects don't read the actual data when `Ascot` object is initialized to keep it light-weight.
The `read` method reads the raw data from the HDF5 file but for post-processing purposes there are better tools which are introduced later.

In [ ]:
a5.data.options.active.get_qid()
a5.data.options.active.get_date()
a5.data.options.active.get_name()

a5.data.options.active.set_desc("New tag")
# The tag was updated
a5.data.options.NEW.get_desc()
a5.data.options.active.activate() # Set group as active
#a5.data.options.active.destroy() # This would remove the data from the HDF5 file
a5.data.options.active.read()

Most of what has been said is true for the result groups as well.
Result groups also hold direct references to inputs.
Again, `ls()` shows overview of the group's contents.

In [ ]:
a5.data.HELLO.get_desc() # Recall we set run description ascot5_main --d="Hello world!"
a5.data.HELLO.bfield.get_desc() # Inputs used in a run can be referenced like this
info = a5.data.HELLO.ls(show=True)

This concludes the tutorial on how the file is organized and accessed.
For input generation and post-processing, the `Ascot` object, its `data` attribute, and the result groups are mostly relevant.

<a id='ascotpy'></a>

## Python interface to libascot.so

Many of the Python tools in `a5py` make use of the `libascot.so` shared library that provides direct access to same functions that `ascot5_main` uses to trace markers and interpolate inputs.
The `Ascot` object automatically initializes the interface to `libascot` via `ascotpy` package provided that the library has been compiled.

However, inputs must be initialized and freed manually. Here's an example on how to initialize magnetic field input.

In [ ]:
from a5py import Ascot

a5 = Ascot("ascot.h5") # Use the same file as in the previous tutorials
a5.input_init(bfield=True) # Initialize active bfield input

# To initialize specific input, provide its QID as a string. Since a bfield is already initialized,
# use switch=True to switch input or else exception is raised.
#a5.input_init(bfield="1234567890", switch=True)

a5.input_initialized() # Shows what inputs are currently initialized

Routines that require the Python interface will raise an exception if required input has not been initialized before the routine was called.
Since magnetic field is now initialized, we can safely interpolate and plot it.
Once you no longer need the specific data, you can deallocate it to free some memory.
Note that marker and options input cannot be initialized (here).

You can ignore the warnings below.
They just inform you in what units the functions expect the arguments to be in.
The units in ``a5py`` are implemented via ``unyt`` package; see the documentation for details.

In [ ]:
import matplotlib.pyplot as plt

psi, rho = a5.input_eval(6.2, 0, 0, 0, "psi", "rho")
print("psi = %.2f, rho = %.2f" % (psi, rho))

ax = plt.figure().add_subplot(1,1,1)
a5.input_plotrz(np.linspace(4,8,50), np.linspace(-4, 4, 100), "psi", axes=ax)
plt.show(block=False)

a5.input_free(bfield=True) # Deallocates just the magnetic field input
a5.input_free()            # Deallocates all inputs (except markers and options)

<a id='inputgen'></a>

## Input generation

ASCOT5 is modular when it comes to inputs.
Several different implementations of magnetic field, electric field, etc. exist.
When planning a study, check from the documentation (see `a5py.ascot5io`) what kind of inputs would serve you and what kind of data those need.
The required data is listed in the `write_hdf5` function corresponding to that input.

Once that is decided, there are two ways to proceed.
Templates for different kind of inputs can be found in `a5py.templates`.
Some of the templates import data from external sources, e.g. EQDSK, to ASCOT5 and using those is strongly recommended.

If no suitable template exists, one must generate the arguments for the `write_hdf5` function themselves.

Running `ascot5_main` requires that all input parents (`bfield`, `efield`, `plasma`, `wall`, `neutral`, `boozer`, `mhd`, `marker`, `options`) have at least one input present.
Some of these are actually rarely used in a simulation and for those it is sufficient to provide dummy data.

No matter how or what input is created, all is done via `create_input` method.

In [ ]:
from a5py import Ascot
a5 = Ascot("ascot.h5")

# Call explicitly E_TC.write_hdf5 function that requires exyz as a parameter
a5.data.create_input("E_TC", exyz=np.array([0,0,0]), activate=True, desc="Zero electric field")

# Use template
a5.data.create_input("bfield analytical iter circular")

<a id='postprocess'></a>

## Post-processing

Simulations are post-processed by using the corresponding run group in `data`.
Run groups provide access to the data, supports evaluation of quantities derived from the data, and host many routines to export or plot the data.

In [ ]:
from a5py import Ascot
a5 = Ascot("ascot.h5")

# Get final (R,z) coordinates of all markers that hit the wall
r,z = a5.data.active.getstate("r", "z", state="end", endcond="wall", ids=None)

# Plot (time, energy) of confined marker orbits
ax = plt.figure().add_subplot(1,1,1)
a5.data.active.plotorbit_trajectory("time", "ekin", endcond="not wall", axes=ax)
plt.show(block=False)

# Summarize simulation
a5.data.active.getstate_markersummary()

# Visualize losses

# Etc... see the documentation of RunGroup for details

<a id='simulation'></a>

## Live simulations

The Python interface to `libascot.so` provides a way to run simulations directly from Python.
These "live" simulations are equivalent to those run via `ascot5_main` except that the markers, options, and results are not stored in the HDF5 file.
These simulations are convenient to use, but the main intention is to use them for post-processing or light simulations on a desktop.

Running live simulations requires that you have the inputs (excluding markers and options) present in the HDF5 file.

In [ ]:
from a5py import Ascot
a5 = Ascot("ascot.h5")

# This method initializes and "packs" inputs in a single array. No input data can be freed while the
# data is packed.
a5.simulation_initinputs()

# Marker input can be anything but here we just use the on ascot.h5
mrk = a5.data.marker.active.read()
a5.simulation_initmarkers(**mrk)

# Options input can also be anything but here we just use the on ascot.h5
opt = a5.data.options.active.read()
a5.simulation_initoptions(**opt)

print("Input initialized")

Running the live simulation returns a `VirtualRun` object which in many ways behaves similarly as the `RunGroup` introduced earlier.

In [ ]:
vrun = a5.simulation_run()

# Get final (R,z) coordinates of all markers that hit the wall
r, z = vrun.getstate("r", "z", state="end", endcond="wall", ids=None)
print(r,z)

# Plot (time, energy) of confined marker orbits
ax = plt.figure().add_subplot(1,1,1)
vrun.plotorbit_trajectory("time", "ekin", endcond="not wall", axes=ax)
plt.show(block=True)

# Summarize simulation
vrun.getstate_markersummary()

# Visualize losses

# Etc... see the documentation of RunGroup for details

To rerun the code, free the simulation output.
Once it is freed, the previous `VirtualRun` becomes an empty shell and it is no longer usable.
Once you have had enough fun, the markers should be freed and inputs unpacked and deallocated.

In [ ]:
a5.simulation_free(diagnostics=True)
a5.simulation_run()

a5.simulation_free(inputs=True, markers=True, diagnostics=True)

## Need help?

1. Ask in our ASCOT5 Slack channel.

2. If you have an issue to report, use the GitHub issue tracker.
   For bugs, state which version/branch you are using and try to provide the HDF5 file.

3. Join one of our "weekly" meetings to present your research and discuss any issues.